# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. 
Требуется разобраться:
 * влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.

**Цель исследования** - проверить гипотезы:
1. Семейное положение "женат/замужем/гражданский брак" способсвтует отсутствию задолженности по возврату кредита
2. Наличие детей в семье способствтует отсутствию задолженности по возврату кредита

**Ход исследования**

Данные о поведении пользователей содержатся в файле (Путь к файлу: /datasets/data.csv). О качестве данных ничего не известно. Поэтому перед проверкой гипотез проведем обзор данных. 

Проверим данные на ошибки и оценим их влияние на исследование. На этапе предобработки исправим самые критичные ошибки данных.
 
Таким образом, исследование содержит три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.


Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Обзор данных

In [1]:
from IPython.display import display
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/data.csv')

In [3]:
df.head(10) #печать первых 10 строк таблицы данных

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о таблице:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итого: 21525 строк. Тип данных - int64 (целые числа), float64	(числа с плавающей точкой), object(текстовые или смешанные числовые и нечисловые значения).

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.




**Выводы**

В каждой строке таблицы - данные о заемщике.

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных.

Чтобы двигаться дальше, устраним проблемы в данных.


## Предобработка данных
- исключим пропуски
- проверим данные на дубликаты и аномалии
- проверим правильность типов данных

###  Заполнение пропусков

Количество значений в столбцах *'days_employed'* и *'total_income'* различается. Значит, в данных есть пропущенные значения.

Cтолбцs *'days_employed'* и *'total_income'* имеют тип данных 'float64' - вещественные числа.

Посчитаем долю пропущенных значений:

In [5]:
print('Доля пропущенных значений в столбце общего трудового стажа в днях:', 
      int(df['days_employed'].isna().sum()/df['children'].count()*100),'%')
print('Доля пропущенных значений в столбце ежемесячного дохода:',
      int(df['total_income'].isna().sum()/df['children'].count()*100),'%')

Доля пропущенных значений в столбце общего трудового стажа в днях: 10 %
Доля пропущенных значений в столбце ежемесячного дохода: 10 %


**Возможные причины пропуска данных:**
1. Отказ от предоставления сведений
2. Изначально неверно проставленное значение (человеческий фактор) - допущены символы/пропуски, которые обработаны программой в NaN или пробел

Попробуем заполнить выявленные пропуски медианным зачением. 

В данном случае такое решение является наиболее приемлемым, поскольку данные имеют сильный разброс по значениям.

Это легко проверить, найдя минимальное и максимальное значение по столбцу *'days_employed'*:

In [6]:
print('Минимальный трудовой стаж в днях:', df['days_employed'].abs().min())
print('Максимальный трудовой стаж в днях:', df['days_employed'].abs().max())


Минимальный трудовой стаж в днях: 24.14163324048118
Максимальный трудовой стаж в днях: 401755.40047533


*//Обратим внимание, что данные из файла по столбцу 'days_employed' были предоставлены в том числе с отрицательными значениями, что скорее всего также является ошибкой при заполнении. Поэтому дополнительно применяем метод abs()//*

То же самое производим касательно столбца *'total_income'*:

In [7]:
print('Минимальный ежемесячный доход:', df['total_income'].abs().min())
print('Максимальный ежемесячный доход:', df['total_income'].abs().max())


Минимальный ежемесячный доход: 20667.26379327158
Максимальный ежемесячный доход: 2265604.028722744


Таким образом, в подобном случае (когда велика разница минимальнойго и максимального значения) логичнее использовать медианные значения для заполнения пропусков по столбцам:

In [8]:
days_employed_median = df['days_employed'].abs().median()
total_income_median = df['total_income'].abs().median()

df['days_employed'] = df['days_employed'].fillna(days_employed_median)
df['total_income'] = df['total_income'].fillna(total_income_median)

In [9]:
print(df.isna().sum())# проверка результатов - подсчёт пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64




!ПРОПУСКИ ЗАПОЛНЕНЫ!

### Изменение типа данных

Как уже отмечалось выше, некоторые значения в столбце *'days_employed'* представлены как отрицательные числа.

Такая ошибка возможна в случае предоставления данных, например, в формате "Общий стаж работы -2 года 14 месяцев".

Переведем все значения в удобный формат (дополнительно применим метод и к столбцу *'total_income'*):

In [10]:
df['days_employed'] = df['days_employed'].abs()
df['total_income'] = df['total_income'].abs()
display(df.head(10)) #проверим результат

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Как видно из полученной таблицы, после применения метода abs(), значения в столбцах *'days_employed'* и *'total_income'* имеет тип данных 'float64'.

Для упрощения анализа изменим тип данных на целочисленные значения int64:

In [11]:
df['days_employed'] = df['days_employed'].fillna(value = days_employed_median).astype('int')
df['total_income'] = df['total_income'].fillna(value = days_employed_median).astype('int')
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


###  Удаление дубликатов и аномальных значений.

In [13]:
df.duplicated().sum()

54

Просмотрим уникальные названия в столбцах *'education', 'family_status', 'income_type', 'purpose', 'gender'*  в алфавитном порядке:

In [15]:
display(sorted(df['education'].unique()))

['ВЫСШЕЕ',
 'Высшее',
 'НАЧАЛЬНОЕ',
 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
 'Начальное',
 'Неоконченное высшее',
 'СРЕДНЕЕ',
 'Среднее',
 'УЧЕНАЯ СТЕПЕНЬ',
 'Ученая степень',
 'высшее',
 'начальное',
 'неоконченное высшее',
 'среднее',
 'ученая степень']

In [16]:
display(sorted(df['family_status'].unique()))


['Не женат / не замужем',
 'в разводе',
 'вдовец / вдова',
 'гражданский брак',
 'женат / замужем']

In [17]:
display(sorted(df['income_type'].unique()))


['безработный',
 'в декрете',
 'госслужащий',
 'компаньон',
 'пенсионер',
 'предприниматель',
 'сотрудник',
 'студент']

In [18]:
display(sorted(df['purpose'].unique()))

['автомобили',
 'автомобиль',
 'высшее образование',
 'дополнительное образование',
 'жилье',
 'заняться высшим образованием',
 'заняться образованием',
 'на покупку автомобиля',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'на проведение свадьбы',
 'недвижимость',
 'образование',
 'операции с жильем',
 'операции с коммерческой недвижимостью',
 'операции с недвижимостью',
 'операции со своей недвижимостью',
 'покупка жилой недвижимости',
 'покупка жилья',
 'покупка жилья для сдачи',
 'покупка жилья для семьи',
 'покупка коммерческой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'получение высшего образования',
 'получение дополнительного образования',
 'получение образования',
 'приобретение автомобиля',
 'профильное образование',
 'ремонт жилью',
 'свадьба',
 'свой автомобиль',
 'сделка с автомобилем',
 'сделка с подержанным автомобилем',
 'строительство жилой недвижимости',
 'строительство недвижимости',
 'строительство собственной недв

In [19]:
display(sorted(df['gender'].unique()))

['F', 'M', 'XNA']

Довольно странное значение 'XNA' для указания пола. Посчитаем количество таких строк:

In [20]:

print(df[df['gender']=='XNA']['gender'].count())

1


Единственное совпадение, удаление которого не повлияет на результат:

In [21]:
df=df[df['gender']!='XNA']

In [22]:
df['education'] = df['education'].str.lower() #переведем прописные буквы в строчные

display(sorted(df['education'].unique())) #проверим уникальные значения столбца education

['высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']

In [23]:
display(sorted(df['children'].unique())) #проверим также и столбец children

[-1, 0, 1, 2, 3, 4, 5, 20]

Видно, что в данные попали довольно странные значения (-1) и 20. 

Значение(-1) заменим на 1, применив метод abs() ко всему столбцу. Знаечние 20 проверим на количество совпадений в таблице:

In [24]:
df['children']=df['children'].abs()

In [25]:
print(df[df['children']==20]['children'].count())


76


In [26]:
print(df[df['children']==20]['children'].count()/df['children'].count()*100) 
#посчитаем процент таких данных со значением 20 в таблице

0.35309422040512917


Процент данных очень мал. Поэтому удалим данные, природа ошибки по которым не ясна (возможно пользователь вводил 0, возможно 2, но при этом задевал соседнюю кнопку).

In [27]:
df=df[df['children']!=20]#создадим новую таблицу, без учета данных по семьям с детьми=20

df.info()#проверим общую информацию о таблице

print(sorted(df['children'].unique()))#проверим уникальные значения по столбцу children

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21448 entries, 0 to 21524
Data columns (total 12 columns):
children            21448 non-null int64
days_employed       21448 non-null int64
dob_years           21448 non-null int64
education           21448 non-null object
education_id        21448 non-null int64
family_status       21448 non-null object
family_status_id    21448 non-null int64
gender              21448 non-null object
income_type         21448 non-null object
debt                21448 non-null int64
total_income        21448 non-null int64
purpose             21448 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB
[0, 1, 2, 3, 4, 5]


Проведем подсчет явных дубликатов:

In [28]:
df.duplicated().sum()

71

И удалим их (с удалением старых индексов и формированием новых)

In [29]:
df=df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum() #проверка на отсутствие дубликатов

0

**Выводы**

Предобработка обнаружила три проблемы в данных:

- наличие пропусков
- наличие дубликатов и аномалий
- излишний тип данных

 

Пропущенные значения заменили на медианные значения. 

Дубликаты откорректировали и лишние из них удалили, чтобы исследование было более точным.

Нашли аномалии и также удалили их.

Тип данных привели к целочисленным значениям.

###  Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

При изучении таблицы видно, что столбцы *'education'* и *'education_id'* можно сопоставит между собой:

In [40]:
education_categories=df[['education','education_id']] #создадим датафрейм из нужных столбцов

display(education_categories.drop_duplicates().reset_index(drop=True).sort_values('education_id')) #сопоставим столбцы



,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


То же самое проведем для столбцов *'family_status'* и *'family_status_id'*.

In [41]:
family_categories=df[['family_status','family_status_id']]

display(family_categories.drop_duplicates().reset_index(drop=True).sort_values('family_status_id'))


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Из такой таблицы можно заметить, что 2 категории (женат/замужем и гражданский брак) можно объединить в дополнительную категорию **'couple'** (пара, неодинокие). 

И остальные 3 категории (вдовец/вдова, в разводе и не замужем/не женат) можно обединить в дополнительную категорию **'single'**(одинокие).

 Для этого напишем функцию, которая присваивает категории couple или single по столбцу family_status_id:

In [42]:
def category_family_group(status):
    if status==0 or status==1:
        return 'couple'
   
    return 'single'

df['category_family']=df['family_status_id'].apply(category_family_group)#обращаемся к столбцу family_status_id,
#у корого имеется 5 значений в соответствии с ранее выведенным словарем и вызываем функцию

Теперь имея такие сопоставления ("словари"), можно удалить из исходной таблицы столбцы *'family_status'*, *'education'* и *'family_status_id'*:

In [43]:
df = df.drop('family_status', axis = 1)
df = df.drop('family_status_id', axis = 1)
df = df.drop('education',axis = 1)
display(df.head(15))

,children,days_employed,dob_years,education_id,gender,income_type,debt,total_income,purpose,category_family
0,1,8437,42,0,F,сотрудник,0,253875,покупка жилья,couple
1,1,4024,36,1,F,сотрудник,0,112080,приобретение автомобиля,couple
2,0,5623,33,1,M,сотрудник,0,145885,покупка жилья,couple
3,3,4124,32,1,M,сотрудник,0,267628,дополнительное образование,couple
4,0,340266,53,1,F,пенсионер,0,158616,сыграть свадьбу,couple
5,0,926,27,0,M,компаньон,0,255763,покупка жилья,couple
6,0,2879,43,0,F,компаньон,0,240525,операции с жильем,couple
7,0,152,50,1,M,сотрудник,0,135823,образование,couple
8,2,6929,35,0,F,сотрудник,0,95856,на проведение свадьбы,couple
9,0,2188,41,1,M,сотрудник,0,144425,покупка жилья для семьи,couple


### Категоризация дохода.

Для столбца *'total_income'* попробуем провести категоризацию доходов. 

Для этого напишем функцию, которая назначает категории в зависимости от размера ежемесячного дохода:

In [44]:
def category_group(income):
    if 0<=income<=30000:
        return 'E'
    if 30001<=income<=50000:
        return 'D'
    if 50001<=income<=200000:
        return 'C'
    if 200001<=income<=1000000:
        return 'B'
    return 'A'


df['total_income_category']=df['total_income'].apply(category_group)#применим функцию к столбцам нашей таблицы 
#и создадим столбец выведенных категорий

Имея такую категоризацию, можно удалить из  таблицы столбец *'total_income'*:

In [45]:
df = df.drop('total_income',axis = 1)
display(df.head(5))

,children,days_employed,dob_years,education_id,gender,income_type,debt,purpose,category_family,total_income_category
0,1,8437,42,0,F,сотрудник,0,покупка жилья,couple,B
1,1,4024,36,1,F,сотрудник,0,приобретение автомобиля,couple,C
2,0,5623,33,1,M,сотрудник,0,покупка жилья,couple,C
3,3,4124,32,1,M,сотрудник,0,дополнительное образование,couple,B
4,0,340266,53,1,F,пенсионер,0,сыграть свадьбу,couple,C


### Шаг 2.7. Категоризация целей кредита.

В соответствии с ранее выведенными уникальными значениями столбца *'purpose'*, так же проведем категоризацию целей кредита.

Проанализировав список уникальных значений,напишем функцию, которая в зависимости от целей кредита присваивает ему определенную категорию:

In [46]:
def purpose_group(purpose):
        
    if purpose.__contains__('авто'):
        return 'операции с автомобилем' 
    if purpose.__contains__('образ'):
        return 'получение образования' 
    if purpose.__contains__('жиль') or purpose.__contains__('недвиж'):
        return 'операции с недвижимостью' 
    return 'проведение свадьбы'
    
df['purpose_category']=df['purpose'].apply(purpose_group)#применим функцию к столбцам нашей таблицы 
#и создадим столбец выведенных категорий

Имея такую категоризацию, можно удалить из  таблицы столбец *'purpose'*:

In [47]:
df = df.drop('purpose',axis = 1)
display(df.head(5))

,children,days_employed,dob_years,education_id,gender,income_type,debt,category_family,total_income_category,purpose_category
0,1,8437,42,0,F,сотрудник,0,couple,B,операции с недвижимостью
1,1,4024,36,1,F,сотрудник,0,couple,C,операции с автомобилем
2,0,5623,33,1,M,сотрудник,0,couple,C,операции с недвижимостью
3,3,4124,32,1,M,сотрудник,0,couple,B,получение образования
4,0,340266,53,1,F,пенсионер,0,couple,C,проведение свадьбы


**Выводы**

В конечном итоге мы получили таблицу, в которой выделены категории по нескольким параметрам. 

Теперь таблица удобна для более глубокого анализа.

In [95]:
#children = df.groupby(by = 'children')['debt'].count()
#display(children)
#важно еще проверить, снижается ли количество заемщиков в зависимости от увеличения количества детей

## Проверка гипотез


### Влияние  детей на наличие задолженности по выплате кредита

Проверим, как влияет наличие детей на наличие задолженности по выплате кредитов. 

Для этого проведем анализ по двум параметрам: наличие детей и наличие задолженности:

In [30]:
df_pivot = df.pivot_table(columns='children', values='debt', aggfunc=['sum','count','mean'])
print(df_pivot)

           sum                      count                              mean  \
children     0    1    2   3  4  5      0     1     2    3   4  5         0   
debt      1063  445  194  27  4  0  14090  4855  2052  330  41  9  0.075444   

                                                       
children         1         2         3         4    5  
debt      0.091658  0.094542  0.081818  0.097561  0.0  


Из сводной талицы видно, что большая часть задолженностей приходится на заемщиков, которые не имеют детей.

Проверим долю должников БЕЗ ДЕТЕЙ среди всех должников:

In [150]:
all_debt = df[df['debt']==1]['debt'].count()#посчитали общее количесвто должников
print(f'Доля заемщиков БЕЗ ДЕТЕЙ, которые имеют задолженность: {1063/all_debt:.2%}')

Доля заемщиков БЕЗ ДЕТЕЙ, которые имеют задолженность: 61.34%


**Из анализа видно, что 61% всех заемщиков, которые имеют задолженность по выплатам кредита, - это семьи без детей!!!**

In [38]:
df.pivot_table(index = 'children', values='debt', aggfunc=['sum','count','mean'])

,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14090,0.075444
1,445,4855,0.091658
2,194,2052,0.094542
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


### Влияние  семейного положения на наличие задолженности по выплате кредита


Проверим, как влияет семейное положение на наличие задолженности по выплате кредитов.

Проведем анализ по двум параметрам: семейное положение и наличие задолженности:

In [155]:
df_pivot = df.pivot_table(columns='category_family', values='debt', aggfunc=['sum','count','mean'])
print(df_pivot)

                   sum         count             mean          
category_family couple single couple single    couple    single
debt              1313    420  16428   4949  0.079925  0.084866


Из сводной талицы видно, что большая часть задолженностей приходится на неодиноких заемщиков.

Проверим долю НЕОДИНОКИХ должников  среди всех должников:

In [168]:
all_debt = df[df['debt']==1]['debt'].count()#посчитали общее количесвто должников
print(f'Доля НЕОДИНОКИХ заемщиков, которые имеют задолженность: {1313/all_debt:.2%}')

Доля НЕОДИНОКИХ заемщиков, которые имеют задолженность: 75.76%


**Из анализа видно, что 75% заемщиков, которые имеют задолженность по выплатам кредита, это неодинокие заемщики!**

In [274]:
#family_status = df.groupby(by = 'category_family')['debt'].count()
#display(family_status)
#важно еще проверить, снижается ли количество заемщиков в зависимости от семейного статуса

### Влияние уровня дохода на наличие задолженности по выплате кредита


In [164]:
df_pivot = df.pivot_table(columns='total_income_category', values='debt', aggfunc=['sum','count','mean'])
print(df_pivot)

                      sum                   count                        mean  \
total_income_category   A    B     C   D  E     A     B      C    D   E     A   
debt                    2  354  1354  21  2    25  5020  15960  350  22  0.08   

                                                           
total_income_category         B         C     D         E  
debt                   0.070518  0.084837  0.06  0.090909  


Из сводной талицы видно, что большая часть задолженностей приходится на заемщиков с доходом 50-200 тысяч рублей.

Проверим долю должников с доходом 50-200 тыс руб среди всех должников:


In [171]:
all_debt = df[df['debt']==1]['debt'].count()#посчитали общее количесвто должников
print(f'Доля заемщиков СО СРЕДНИМ ДОХОДОМ 50-200 ТЫС РУБ, которые имеют задолженность: {1354/all_debt:.2%}')

Доля заемщиков СО СРЕДНИМ ДОХОДОМ 50-200 ТЫС РУБ, которые имеют задолженность: 78.13%


**Из анализа видно, что 78% заемщиков, которые имеют задолженность по выплатам кредита, это заемщики с доходом 50-200 тысяч рублей!**

In [278]:
#income_status = df.groupby(by = 'total_income_category')['debt'].count()
#display(income_status)
#важно еще проверить, снижается ли количество заемщиков в зависимости от уровня дохода

In [277]:
purpose_status = df.groupby(by = 'purpose_category')['debt'].count()
display(purpose_status)
#важно еще проверить, снижается ли количество заемщиков в зависимости от семейного статуса

purpose_category
операции с автомобилем       4292
операции с недвижимостью    10778
получение образования        3999
проведение свадьбы           2326
Name: debt, dtype: int64

### Влияние целей кредита на наличие задолженности по выплате 


In [166]:
df_pivot = df.pivot_table(columns='purpose_category', values='debt', aggfunc=['sum','count','mean'])
print(df_pivot)

                                    sum                           \
purpose_category операции с автомобилем операции с недвижимостью   
debt                                401                      780   

                                                           \
purpose_category получение образования проведение свадьбы   
debt                               369                183   

                                  count                           \
purpose_category операции с автомобилем операции с недвижимостью   
debt                               4290                    10774   

                                                           \
purpose_category получение образования проведение свадьбы   
debt                              3998               2315   

                                   mean                           \
purpose_category операции с автомобилем операции с недвижимостью   
debt                           0.093473                 0.072397   

                 

Из сводной талицы видно, что большая часть задолженностей приходится на заемщиков с кредитом на жилье.

Проверим долю должников с жилищным кредитом среди всех должников:


In [172]:
all_debt = df[df['debt']==1]['debt'].count()#посчитали общее количесвто должников
print(f'Доля заемщиков ПО КРЕДИТУ НА ЖИЛЬЕ, которые имеют задолженность: {780/all_debt:.2%}')

Доля заемщиков ПО КРЕДИТУ НА ЖИЛЬЕ, которые имеют задолженность: 45.01%


**Из анализа видно, что 45% заемщиков, которые имеют задолженность по выплатам кредита, это заемщики по кредиту на жилье.**

In [48]:
df.pivot_table(index='purpose_category', values='debt', aggfunc=['sum','count','mean'])

,sum,count,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,401,4290,0.093473
операции с недвижимостью,780,10774,0.072397
получение образования,369,3998,0.092296
проведение свадьбы,183,2315,0.079050


### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

##### Вывод 1:

Заметна зависимость между показателями количества детей и возврата кредита в срок: больше всего заемщиков среди бездетных заемщиков. 

С ростом количества детей показатель снижается - чем больше детей, тем меньше процент должников. 

Однако стоит отметить, что с ростом количества детей количество заемщиков ПАДАЕТ, что так же сказывается на выводе!

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

##### Вывод 2:

Определенно найдена зависимость - больше заемщиков, которые имеют задолженность по выплатам кредита, это неодинокие заемщики!!!

Однако стоит отметить, что с неодиноких заемщиков меньше, что так же сказывается на выводе!

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

##### Вывод 3:

Определенно найдена зависимость - больше заемщиков, которые имеют задолженность по выплатам кредита, это заемщики со средним уровнем дохода!!!

Однако стоит отметить, что  таких заемщиков гораздо больше, что так же сказывается на выводе!

##### Вопрос 4:

Есть ли зависимость между целями кредита и возвратом кредита в срок?

##### Вывод 4:

Определенно найдена зависимость - больше заемщиков имеют задолженность по выплатам кредита на жилье.

Однако стоит отметить, что  таких заемщиков гораздо больше, что так же сказывается на выводе!

## Общий вывод:

Согласно представленным входным данным проведен анализ базы заемщиков. 

Для начала был выявлен ряд несоответствий, пропуски данных, неверные значения и тд. Такие моменты были устранены. 

Далее проверили несколько гипотез и установили:

1. На возврат кредита в срок влияют:
- наличие детей и их количество;
- семейное положение;
- уровень дохода;
- цель кредита.

Кредитному отделу банка рекомендуется обращать внимание на наличие детей, тк задолженность по кредиту чаще встречается у семей без детей. 

Также более привлекательны в качестве заемщиков одинокие люди. 

Заемщики со средним уровнем дохода в любом случае будут чаще выступать должниками. 

А кредиты на жилье не выплачиваются чаще остальных.

При рассмотрении кредитным отделом банка привлекательности клиента-заемщика рекомендуется учитывать данные выводы и включить их в оценку способности потенциального заёмщика вернуть кредит банку (как вариант предлагать разные программы кредитования для разных категорий заемщкиов и программы кредитования). 


Дополнительно стоит отметить, что анализ проводился на разном количестве заемщиков с теми или иными параметрами. Для более точного вывода можно провести анализ на одинаковом количестве заемщиков (например, 1500 заемщиков без детей и 1500 заемщиков с 1/2/3 детьми), что будет более показательным для построения модели кредитного скоринга.